
# OpenVINO Model Benchmarking

This notebook benchmarks the performance of a simple and optimized UNet model on both CPU and GPU using OpenVINO. 
It measures inference time and plots the comparison results.

---
### Steps:
1. Preprocess an input image to match the model requirements.
2. Run inference on CPU and GPU for both simple and optimized models.
3. Collect metrics (inference time).
4. Visualize results using a bar graph.

---


In [ ]:

from openvino.runtime import Core
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt


In [ ]:

# Paths to OpenVINO model files
SIMPLE_MODEL_XML = "path/to/simple_model.xml"
OPTIMIZED_MODEL_XML = "path/to/optimized_model.xml"

# Initialize OpenVINO Core
ie = Core()


In [ ]:

def preprocess_image(image_path, target_shape=(480, 368)):
    """Preprocess input image."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}.")
    _, thresholded = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY)
    resized = cv2.resize(thresholded, target_shape)
    rgb_image = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)
    normalized = rgb_image / 255.0
    transposed = normalized.transpose(2, 0, 1)
    return np.expand_dims(transposed, axis=0).astype(np.float32)


In [ ]:

def run_inference(image_path, model_path, device="CPU"):
    """Run model inference."""
    model = ie.read_model(model=model_path)
    compiled_model = ie.compile_model(model=model, device_name=device)
    input_tensor = preprocess_image(image_path)
    
    # Measure inference time
    start_time = time.time()
    results = compiled_model([input_tensor])
    inference_time = time.time() - start_time
    
    output = next(iter(results.values()))
    segmentation_mask = np.argmax(output, axis=1).squeeze().astype(np.uint8)
    
    return inference_time, segmentation_mask


In [ ]:

def benchmark_models(image_path):
    """Benchmark both models on CPU and GPU."""
    configurations = [
        ("Simple Model", SIMPLE_MODEL_XML, "CPU"),
        ("Simple Model", SIMPLE_MODEL_XML, "GPU"),
        ("Optimized Model", OPTIMIZED_MODEL_XML, "CPU"),
        ("Optimized Model", OPTIMIZED_MODEL_XML, "GPU"),
    ]
    metrics = []
    
    for name, model_path, device in configurations:
        time_taken, _ = run_inference(image_path, model_path, device=device)
        metrics.append((name, device, time_taken))
        print(f"{name} on {device} - Time: {time_taken:.4f}s")
    
    return metrics


In [ ]:

def plot_results(metrics):
    """Plot benchmark results."""
    labels = [f"{name}-{device}" for name, device, _ in metrics]
    times = [time for _, _, time in metrics]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(labels, times, color=["blue", "skyblue", "green", "lightgreen"], alpha=0.7)
    
    for bar, time in zip(bars, times):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02, f"{time:.2f}s", ha='center', fontsize=10)
    
    ax.set_ylabel("Inference Time (s)")
    ax.set_title("Inference Time Comparison: Simple vs. Optimized Models on CPU and GPU")
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:

# Example usage:
# Provide the path to an image for benchmarking
image_path = "path/to/test_image.jpg"

# Benchmark models and collect metrics
metrics = benchmark_models(image_path)

# Plot the results
plot_results(metrics)
